In [16]:
from jupyter_cadquery import set_defaults, set_sidecar
from jupyter_cadquery.cadquery import (PartGroup, Part, show as show_object)
from mkr3d.commands import post

set_defaults(theme="dark")
set_sidecar("CadQuery", init=True)
import cadquery as cq


/home/semartin/miniconda3/envs/jcq22/lib/python3.8/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [17]:
BAT_MOUNT_DIM = [99, 70, 20]
BAT_MOUNT_FILLET = [15, 10]
BAT_MOUNT_HOLES = [(-39,0), (29.5, -24.5), (29.5, 24.5)]
POST_HOLE_DIA = 2.5
ARM_DIM = [70,20]
ARM_HOLE_DIA = 5
BAR_HOLE_DIA = 5.2
WIRE_HOLE_DIA = 4.2
THICK = 5
NARMS = 3

bar = cq.Workplane().box(200, 20, 20).translate((0,0,-10))

bar_clip = (
    cq.Workplane().box(BAT_MOUNT_DIM[0], 20.2+2*THICK, 20).translate((0,0,-10))
    .cut(bar)
    .faces(">Y").workplane() 
    .pushPoints([(BAT_MOUNT_DIM[0]/3, -10), (-BAT_MOUNT_DIM[0]/3, -10)])
    .hole(BAR_HOLE_DIA)
)

inside = (
    cq.Workplane().box(BAT_MOUNT_DIM[0]-2*THICK, BAT_MOUNT_DIM[1]-2*THICK, BAT_MOUNT_DIM[2], centered=[True, True, False])
    .translate((0,0,THICK))
    .edges("<X and |Z").fillet(BAT_MOUNT_FILLET[0])
    .edges(">X and |Z").fillet(BAT_MOUNT_FILLET[1])
    .union(
        cq.Workplane("XZ").center(-5,0).slot2D(2*WIRE_HOLE_DIA, WIRE_HOLE_DIA).extrude(100, both=True)
        .translate((0,0,2*THICK+WIRE_HOLE_DIA/2))
    )
)
    
part = (
    cq.Workplane().box(*BAT_MOUNT_DIM, centered=[True, True, False])
    .edges("<X and |Z").fillet(BAT_MOUNT_FILLET[0])
    .edges(">X and |Z").fillet(BAT_MOUNT_FILLET[1])
    .union(bar_clip)
    .cut(inside)
    .faces(">Z").workplane().pushPoints(BAT_MOUNT_HOLES)
        .circle(POST_HOLE_DIA/2)
        .rect(POST_HOLE_DIA*6, POST_HOLE_DIA*6)
        .extrude(-BAT_MOUNT_DIM[2])
    #.cut(caster)
)

# PartGroup([Part(p) for p in [bar, part]])
part = part.rotate((0,0,0), (0,1,0),180).rotate((0,0,0),(0,0,1), -90)
part

Done, using side car 'Cadquery'


# Spincaster Battery Mount

This part connects the top half of https://www.thingiverse.com/thing:4497887 to the side of the spincaster, with M5 machine screws.

In [ ]:
show_object(part)

In [18]:
post({
    "battery_mount_8020.stl": part,
}, job="batmount", slice=True, upload=False, queue=False); print("done")

done
